**Table of contents**<a id='toc0_'></a>    
- [Import packages](#toc1_)    
- [Set up working directory](#toc2_)    
- [ROI selection](#toc3_)    
  - [Define parameters](#toc3_1_)    
  - [Define functions](#toc3_2_)    
  - [Define ROI, perform vessel segmentation and mask filtering](#toc3_3_)    
    - [For single channel](#toc3_3_1_)    
      - [Run samples in working directory](#toc3_3_1_1_)    
      - [Generate vessel segmentation masks](#toc3_3_1_2_)    
      - [Vessel metric analysis and save results in Excel](#toc3_3_1_3_)    
    - [For double channel](#toc3_3_2_)    
      - [Run samples in working directory](#toc3_3_2_1_)    
      - [Generate vessel segmentation masks](#toc3_3_2_2_)    
      - [Vessel metric analysis and save results in Excel](#toc3_3_2_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Import packages](#toc0_)

In [ ]:
import sys
import os
import numpy as np
import vessel_metrics as vm
import czifile
from aicspylibczi import CziFile
import matplotlib.pyplot as plt
import cv2
import imageio
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib.colors
from matplotlib.pyplot import rc_context
import math
import seaborn as sns
from scipy.stats import median_abs_deviation 
import glob
import time
import pandas as pd
import openpyxl
import seaborn as sns
import plotly.graph_objects as go
from skimage.measure import label, regionprops
from skimage import color
from skimage.color import label2rgb
from matplotlib.cm import get_cmap
from skimage.morphology import remove_small_objects
from aicsimageio import AICSImage, readers
from scipy.spatial import distance
from skimage.morphology import skeletonize
import traceback
import gc
from scipy.spatial.distance import cdist

# <a id='toc2_'></a>[Set up working directory](#toc0_)

In [ ]:
"""
Define the path to the working directory and the output folder where the 3D volumes will be saved.
Define the name of the files to be processed.
"""
data_path = 'path_to_working_directory'
output_path = 'path_to_output_directory'
file_name = glob.glob(f'{data_path}*.*', recursive = True) # List of file names to process

In [ ]:
"""
Ensure the output directory exists and that files are read correctly.
"""
total_files = len(file_name)
print(f"Wolking forlder contains {total_files} files")

# <a id='toc3_'></a>[ROI selection](#toc0_)

## <a id='toc3_1_'></a>[Define parameters](#toc0_)

In [ ]:
## Please README before running the following code ##

## Parameters that the user should define before running VISTA-Z ROI analysis ##

    # clahe = (int, int) # CLAHE parameters (clipLimit, tileGridSize)
    # ROI_mode = 'square'  or 'polygon'
    # ROI_num = int # Number of ROIs to be analysed per sample
    # channel = int [0,1,2] # Channel number to be analysed per sample (only if multi-channel images) [0]: mCherry, [1]: EGFP, [2]: DAPI, etc.
    # name_start = str # Starting string of the file names to be processed
    # pdf_dpi = int # DPI for saving PDF figures
    # min_vessel_size = int # Minimum vessel size (in pixels) to be thresholded for analysis
    # Vessel segmentation parameters:
        # seg_method = str # Vessel segmentation method: 'meijering', 'frangi', 'sato' or 'jerman'
        # sigma1 = range(int, int, int) # Sigma range for vessel enhancement filter
        # hole_size = int # Maximum hole size to be filled in the vessel mask
        # ditzle_size = int # Maximum size of small objects to be removed from the vessel mask
        # thresh = int # Threshold value to binarize the vessel enhanced image
        # skel_method = str # Skeletonisation method: 'lee' or 'zhang'
    # Branch_dist = int # Maximum distance (in pixels) to consider two vessel branches as connected
    # MAD filtering threshold: Threshold for median absolute deviation (MAD) filtering of vessel metrics
        # threshold_low = float 
        # threshold_high = float
    # Diameter calculation parameters:
        # dist = int # Initial distance from the centerline to search for vessel edges
        # diam = int # Incremental step size to increase the search distance for vessel edges
        # minimum_length = int # Minimum length (in pixels) of cross-lines to be considered for diameter calculation
        # pad_size = int # Padding size (in pixels) around the ROI to avoid edge effects during diameter calculation
        # points = int # Number of points to sample diameters (e.g n_points = 10, every 10th diameter will be measured)
        # window_size = 10
    # Normalisation of vessel metrics to the ROI area:
        # scale = float # Scale factor to normalise vessel metrics (pixels per micron)
        # z = int # Z-step size (microns) between slices in the 3D image stack
    # output_name = str # Name of the output files to be saved (excel file)

In [ ]:
# Image pre-processing and ROI selection parameters
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
ROI_mode = 'square'
ROI_num = 2
channel = 0 
name_start = 'flk1EGFP'
pdf_dpi = 300

# Vessel segmentation parameters
min_vessel_size = 500 # Note: User are reccommended to set this parameter after initial analysis based on median(vessel_length) - std(vessel_length)
seg_method = 'meijering'
sigma1 = range(3, 8, 1)
hole_size = 200
ditzle_size = 50 # Note: Users are reccommended to set hole_size and ditzle_size based on the visualisation of the vessel mask after thresholding. Adjust these parameters to ensure that the vessel mask captures the vessels accurately while minimising noise and artefacts.
thresh = 10 # Note: Users are reccommended to set this parameter based on the visualisation of the vessel enhanced image and the vessel mask after thresholding. Adjust this parameter to ensure that the vessel mask captures the vessels accurately while minimising noise and artefacts.
skel_method = 'lee'

#Branchpoint refinement parameters
Branch_dist = 30 # Note: User are reccommended to set this parameter based median(vessel_length)

# MAD filtering threshold
threshold_low = 1
threshold_high = 3

# Diameter calculation and vessel metrics normalisation parameters
dist = 2
diam = 1   
minimum_length = 25
pad_size = 25
points = 10 # Note: Lower values indicate that more vessel crosslines will be used and therefore using more computational power and processing time
window_size = 10 # 10 is to make an 100µm² area of 10x10. Transform into the low int value to create the scale for the density values
scale = 1.2044
z = 5

# Output file name
output_name = "VISTA-Z_ROI_Results"

# Define empty lists to store all ROI data
all_mipROIs = []
all_grayROIs = []
all_claheROIs= []
all_otsuROIs = []
output_ROI_paths = []
output_ROImask_paths = []
all_coords = []
seg_mask = []

## <a id='toc3_2_'></a>[Define functions](#toc0_)

In [ ]:
def select_roi(image, mode=ROI_mode):
    """
    Interactively select a region of interest (ROI) from a 2D image.
    The ROI can be selected as a rotated rectangle ("square") or a polygon.
    For square selection, the user can rotate and move the box before confirming the ROI.
    
    Args:
        image (ndarray): 2D image from which to select the ROI.
        mode (str): Mode of ROI selection, either 'square' or 'polygon'.
    
    Returns:
        roi_img (ndarray): The selected ROI image.
        mask (ndarray): Boolean mask of the ROI.
        coords (tuple or None): Coordinates of the ROI vertices for polygon selection.
        rect (tuple or None): Coordinates of the ROI (center, size, angle) for rectangle selection.
        image (ndarray): The original image for reference if no ROI is selected.

    """
    image_disp = (image / image.max() * 255).astype(np.uint8) # Convert image to uint8 for display
    roi_img, mask, coords, rect = None, None, None, None 

    if mode == 'square': # ROI selection as square
        from_center = False # Set to True to draw from center
        show_crosshair = True # Set to False to hide crosshair
        x, y, w, h = cv2.selectROI("Select ROI", image_disp, showCrosshair=show_crosshair, fromCenter=from_center) # Create ROI
        if w == 0 or h == 0: # Return full image if no ROI selected
            print("No ROI selected, returning full image")
            roi_img = image # Full image as ROI
            mask = np.ones_like(image, dtype=bool)
            coords = None
            rect = None
            cv2.destroyAllWindows() # Close ROI window
            return roi_img, mask, coords, rect
        angle = 0 # Defines initial angle
        rect = ((x + w / 2, y + h / 2), (w, h), angle) # Define rectangle coordinates
        while True: # Rotate and adjust rectangle
            display_img = cv2.cvtColor(image_disp, cv2.COLOR_GRAY2BGR)
            box = cv2.boxPoints(rect).astype(int) # Get rectangle box points
            cv2.drawContours(display_img, [box], 0, (0, 255, 0), 2) # Draw rectangle contour
            cv2.imshow("Select ROI", display_img) # Display the image with the rectangle
            print("Press Enter to confirm, 'r' to rotate clockwise, 'e' to rotate counter-clockwise,")
            print("'a', 'd', 'w', 's' to move left, right, up, down respectively, and 'c' to cancel.")
            key = cv2.waitKey(0) & 0xFF # Wait for key press
            if key == 13:  # Enter
                break 
            elif key == ord('r'): # Rotate clockwise
                rect = (rect[0], rect[1], (rect[2] + 1) % 360)
            elif key == ord('e'): # Rotate counter-clockwise
                rect = (rect[0], rect[1], (rect[2] - 1) % 360)
            elif key == ord('a'): # Move left
                cx, cy = rect[0]
                rect = ((cx - 10, cy), rect[1], rect[2])
            elif key == ord('d'): # Move right
                cx, cy = rect[0]
                rect = ((cx + 10, cy), rect[1], rect[2])
            elif key == ord('w'): # Move up
                cx, cy = rect[0]
                rect = ((cx, cy - 10), rect[1], rect[2])
            elif key == ord('s'): # Move down
                cx, cy = rect[0]
                rect = ((cx, cy + 10), rect[1], rect[2])
            elif key == ord('c'): # Cancel selection
                cv2.destroyAllWindows() # Close ROI window
                print("Cancelled ROI selection.")
                return image, np.ones_like(image, dtype=bool), None, None # Return full image if cancelled
        cv2.destroyAllWindows() # Close ROI window
        mask_uint8 = np.zeros_like(image, dtype=np.uint8) # Create mask for rotated ROI with zeros
        box = cv2.boxPoints(rect).astype(int) # Get rectangle box points
        cv2.fillPoly(mask_uint8, [box], 1) # Fill the rectangle area in the mask
        mask = mask_uint8.astype(bool) # Convert mask to boolean
        ys, xs = np.where(mask) # Get coordinates of the mask
        ymin, ymax = ys.min(), ys.max() # Get bounding box of the mask
        xmin, xmax = xs.min(), xs.max() 
        roi_img = image[ymin:ymax, xmin:xmax] # Crop ROI from image
        mask = mask[ymin:ymax, xmin:xmax] # Crop mask to ROI
        coords = (xmin, ymin, xmax, ymax) # Define ROI coordinates
    elif mode == 'polygon': # ROI selection as polygon
        points = [] # List to store polygon points
        def click_event(event, x, y, flags, param): # Mouse click to record points
            if event == cv2.EVENT_LBUTTONDOWN: # Left mouse button click
                points.append((x, y)) # Add point to list
                cv2.circle(display_img, (x, y), 3, (255, 0, 0), -1) # Draw point on image
                cv2.imshow("Select ROI", display_img) # Update display with new point
        display_img = cv2.cvtColor(image_disp, cv2.COLOR_GRAY2BGR) 
        cv2.namedWindow("Select ROI", cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_EXPANDED)
        cv2.setWindowProperty("Select ROI", cv2.WND_PROP_TOPMOST, 1)
        cv2.imshow("Select ROI", display_img)
        cv2.setMouseCallback("Select ROI", click_event)
        print("Click points to define polygon. Press 'Enter' when done.") 
        while True: # Wait for user to finish polygon selection
            key = cv2.waitKey(1) & 0xFF # Wait for key press
            if key == 13:  # Enter
                break
            elif key == 27:  # ESC to cancel
                points = []
                break
        cv2.destroyAllWindows() # Close ROI window
        if len(points) < 3: # Return full image if no polygon selected
            print("Polygon not selected, returning full image")
            roi_img = image # Full image as ROI
            mask = np.ones_like(image, dtype=bool)
            coords = None
        else:
            mask_uint8 = np.zeros_like(image, dtype=np.uint8) # Create mask for polygon ROI with zeros
            cv2.fillPoly(mask_uint8, [np.array(points, dtype=np.int32)], 1) # Fill polygon area in the mask
            mask = mask_uint8.astype(bool) # Convert mask to boolean
            ys, xs = np.where(mask) # Get coordinates of the mask
            ymin, ymax = ys.min(), ys.max() # Get bounding box of the mask
            xmin, xmax = xs.min(), xs.max()
            roi_img = image[ymin:ymax, xmin:xmax] # Crop ROI from image
            coords = (xmin, ymin, xmax, ymax) # Get ROI coordinates
    else:
        raise ValueError("mode must be 'square' or 'polygon'") # Raise error for invalid mode
    return roi_img, mask, coords # Return ROI image, mask, and coordinates

def load_and_process_czi_singleROI(file_path, name, output_path, roi = ROI_num):
    """
    Loads and processes a fluorescent image (.czi file) with a single channel, 
    
    Args:
        file_path (str): Full path to the .czi file.
        name (str): File name used to derive the sample ID.
        output_path (str): Directory where output images will be saved.
        roi (int): Number of ROIs to be selected and processed per sample.

    Returns:
        tuple:
            image (ndarray): Raw image data after squeezing dimensions.
            roi_masks (list[ndarray]): List of binary masks for each selected ROI.
            roi_coords (list[tuple]): List of coordinates for each selected ROI.
            mip_image (ndarray): Normalized maximum-intensity projection (MIP) image in uint8.
            gray_image (ndarray): Greyscale MIP image.
            clahe_image (ndarray): CLAHE-enhanced greyscale image.
            otsu_mask (ndarray): Binary mask from Otsu thresholding.
            output_mip_path (str): Path for saving the MIP image.
            output_mask_path (str): Path for saving the Otsu mask image.
            mip_imageROI (list[ndarray]): List of MIP images cropped to each selected ROI.
            roi_grayROI (list[ndarray]): List of greyscale MIP images cropped to each selected ROI.
            roi_claheROI (list[ndarray]): List of CLAHE-enhanced images cropped to each selected ROI.
            roi_otsuROI (list[ndarray]): List of Otsu masks cropped to each selected ROI.
            output_mipROI_paths (list[str]): List of paths for saving each ROI MIP image.
            output_maskROI_paths (list[str]): List of paths for saving each ROI Otsu mask image.
        Returns (None, None, None, None) if processing fails.
    """
    # Defining lists to store ROI data
    roi_masks = []
    roi_coords = []
    mip_imageROI = []
    roi_grayROI = []
    roi_claheROI = []
    roi_otsuROI = []
    output_mipROI_paths = []
    output_maskROI_paths = []
    try:
        sampleID = name # Name of the sample
        with czifile.CziFile(file_path) as czi: 
            data = czi.asarray() # Read confocal image data from .czi file                        
        data_squeezed = np.squeeze(data) # Squeeze data to remove unwanted dimensions/metadata
        image = data_squeezed 
        mip_image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8) # Normalise image to uint8
        mip_image_rgb = cv2.cvtColor(mip_image, cv2.COLOR_GRAY2RGB) # Convert grayscale MIP to RGB
        gray_image = cv2.cvtColor(mip_image_rgb, cv2.COLOR_RGB2GRAY) # Convert RGB MIP to grayscale
        clahe_image = clahe.apply(gray_image) # Apply CLAHE for contrast enhacement
        _, otsu_mask = cv2.threshold(clahe_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) # Otsu's thresholding
        output_mip_path = os.path.join(output_path, f"{sampleID}.tiff") # Define output MIP path
        output_mask_path = os.path.join(output_path, f"{sampleID}_mask.tiff") # Define output mask path
        for i in range(roi): # Loop to selected number of ROIs
            _, mask, coords = select_roi(image, mode=ROI_mode) # Select ROIs
            roi_masks.append(mask) # Store ROI masks
            roi_coords.append(coords) # Store ROI coordinates
        for i, (mask, coords) in enumerate(zip(roi_masks, roi_coords)): # Process each ROI
            if coords is None: # If coordinates are empty
                print(f"No coordinates selected") # No ROI selected, entire image used
                continue
            x1, y1, x2, y2 = coords # Get ROI coordinates
            roi_mip = mip_image[y1:y2, x1:x2] # Crop MIP to ROI
            roi_gray = gray_image[y1:y2, x1:x2] # Crop grayscale MIP to ROI
            roi_clahe = clahe_image[y1:y2, x1:x2] # Crop CLAHE image to ROI
            roi_otsu = otsu_mask[y1:y2, x1:x2] # Crop Otsu mask to ROI
            mip_imageROI.append(roi_mip) # Store ROI MIP
            roi_grayROI.append(roi_gray) # Store ROI grayscale MIP
            roi_claheROI.append(roi_clahe) # Store ROI CLAHE image
            roi_otsuROI.append(roi_otsu) # Store ROI Otsu mask
            output_mipROI_path = os.path.join(output_path, f"{sampleID}_ROI{i}.tiff") # Define output ROI MIP path
            output_maskROI_path = os.path.join(output_path, f"{sampleID}_ROI{i}_mask.tiff") # Define output ROI mask path
            output_mipROI_paths.append(output_mipROI_path) # Store output ROI MIP path
            output_maskROI_paths.append(output_maskROI_path) # Store output ROI mask path
            display_and_save_mip_otsu(roi_gray, roi_clahe, roi_otsu, output_mipROI_path, output_maskROI_path) # Display and save ROI images
        return image, roi_masks, roi_coords, mip_image, gray_image, clahe_image, otsu_mask, output_mip_path, output_mask_path, mip_imageROI, roi_grayROI, roi_claheROI, roi_otsuROI, output_mipROI_paths, output_maskROI_paths
    except Exception as e:
        print(f"Error processing {sampleID}: {e}") # Print error message if processing fails
        return None, None, None, None

def load_and_process_czi_doubleROI(file_path, file_name, output_path, roi = ROI_num, channel = channel):
    """
    Loads and processes a fluorescent image (.czi file) with a single channel, 
    
    Args:
        file_path (str): Full path to the .czi file.
        file_name (str): File name used to derive the sample ID.
        output_path (str): Directory where output images will be saved.
        roi (int): Number of ROIs to be selected and processed per sample.
        channel (int): Channel index. For example mCherry = 0, EGFP = 1, DAPI = 2, etc.

    Returns:
        tuple:
            image (ndarray): Raw image data after squeezing dimensions.
            roi_masks (list[ndarray]): List of binary masks for each selected ROI.
            roi_coords (list[tuple]): List of coordinates for each selected ROI.
            mip_image (ndarray): Normalized maximum-intensity projection (MIP) image in uint8.
            gray_image (ndarray): Greyscale MIP image.
            clahe_image (ndarray): CLAHE-enhanced greyscale image.
            otsu_mask (ndarray): Binary mask from Otsu thresholding.
            output_mip_path (str): Path for saving the MIP image.
            output_mask_path (str): Path for saving the Otsu mask image.
            mip_imageROI (list[ndarray]): List of MIP images cropped to each selected ROI.
            roi_grayROI (list[ndarray]): List of greyscale MIP images cropped to each selected ROI.
            roi_claheROI (list[ndarray]): List of CLAHE-enhanced images cropped to each selected ROI.
            roi_otsuROI (list[ndarray]): List of Otsu masks cropped to each selected ROI.
            output_mipROI_paths (list[str]): List of paths for saving each ROI MIP image.
            output_maskROI_paths (list[str]): List of paths for saving each ROI Otsu mask image.
        Returns (None, None, None, None) if processing fails.
    """
    # Defining lists to store ROI data
    roi_masks = []
    roi_coords = []
    mip_imageROI = []
    roi_grayROI = []
    roi_claheROI = []
    roi_otsuROI = []
    output_mipROI_paths = []
    output_maskROI_paths = []
    
    try:
        sampleID = file_name.split('_MaxInt')[0] # Name of the sample
        with czifile.CziFile(file_path) as czi:
            data = czi.asarray() # Read confocal image data from .czi file              
        data_squeezed = np.squeeze(data) # Squeeze data to remove unwanted dimensions/metadata
        image = data_squeezed[channel] # Retrieve image data from the specified channel
        mip_image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8) # Normalise image to uint8
        mip_image_rgb = cv2.cvtColor(mip_image, cv2.COLOR_GRAY2RGB) 
        gray_image = cv2.cvtColor(mip_image_rgb, cv2.COLOR_RGB2GRAY)
        clahe_image = clahe.apply(gray_image) # Apply CLAHE for contrast enhacement
        _, otsu_mask = cv2.threshold(clahe_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) # Otsu thresholding
        output_mip_path = os.path.join(output_path, f"{sampleID}.tiff") # Define output MIP path
        output_mask_path = os.path.join(output_path, f"{sampleID}_mask.tiff") # Define output mask path
        for i in range(roi): # Loop to selected number of ROIs
            _, mask, coords = select_roi(image, mode=ROI_mode) # Select ROIs
            roi_masks.append(mask) # Store ROI masks
            roi_coords.append(coords) # Store ROI coordinates
        for i, (mask, coords) in enumerate(zip(roi_masks, roi_coords)): # Process each ROI
            if coords is None: # If coordinates are empty
                print(f"No coordinates selected") # No ROI selected, entire image used
                continue
            x1, y1, x2, y2 = coords # Get ROI coordinates
            roi_mip = mip_image[y1:y2, x1:x2] # Crop MIP to ROI
            roi_gray = gray_image[y1:y2, x1:x2] # Crop grayscale MIP to ROI
            roi_clahe = clahe_image[y1:y2, x1:x2] # Crop CLAHE image to ROI
            roi_otsu = otsu_mask[y1:y2, x1:x2] # Crop Otsu mask to ROI
            mip_imageROI.append(roi_mip) # Store ROI MIP
            roi_grayROI.append(roi_gray) # Store ROI grayscale MIP
            roi_claheROI.append(roi_clahe) # Store ROI CLAHE image
            roi_otsuROI.append(roi_otsu) # Store ROI Otsu mask
            output_mipROI_path = os.path.join(output_path, f"{sampleID}_ROI{i}.tiff") # Define output ROI MIP path
            output_maskROI_path = os.path.join(output_path, f"{sampleID}_ROI{i}_mask.tiff") # Define output ROI mask path
            output_mipROI_paths.append(output_mipROI_path) # Store output ROI MIP path
            output_maskROI_paths.append(output_maskROI_path) # Store output ROI mask path
            display_and_save_mip_otsu(roi_gray, roi_clahe, roi_otsu, output_mipROI_path, output_maskROI_path) # Display and save ROI images
        return image, roi_masks, roi_coords, mip_image, gray_image, clahe_image, otsu_mask, output_mip_path, output_mask_path, mip_imageROI, roi_grayROI, roi_claheROI, roi_otsuROI, output_mipROI_paths, output_maskROI_paths
    except Exception as e:
        print(f"Error processing {sampleID}: {e}") # Print error message if processing fails
        return None, None, None, None 
    
def save_plot_as_pdf(fig, output_path, dpi=pdf_dpi):
    """
    Save a Matplotlib figure to a PDF file.

    Args:
        fig (matplotlib.figure.Figure): Figure to save.
        output_path (str): Output path to save the PDF file.
        dpi (int): Resolution to use when saving.

    Returns:
        None
    """
    try:
        fig.savefig(output_path, format='pdf', dpi=dpi, bbox_inches='tight') # Save figure as PDF
        print(f"Plot saved as PDF: {output_path}")
    except Exception as e: # Print error message if saving fails
        print(f"Error saving plot as PDF: {e}")
        
def display_and_save_mip_otsu(gray_image, clahe_image, otsu_mask, output_mip_path, output_mask_path):
    """
    Display MIP, CLAHE, and Otsu mask panels and save images.
    
    Args:
        gray_image (ndarray): Greyscale MIP image.
        clahe_image (ndarray): CLAHE-enhanced greyscale image.
        otsu_mask (ndarray): Binary mask from Otsu thresholding.
        output_mip_path (str): Path for saving the MIP image.
        output_mask_path (str): Path for saving the Otsu mask image.

    Returns:
        None
    """
    fig, axes = plt.subplots(1, 3, figsize=(8, 5)) # Create figure with 3 panels
    axes[0].imshow(gray_image, cmap="gray") # Display MIP
    axes[0].set_title("Maximum Intensity Projection (MIP)")
    axes[0].axis("off")
    axes[1].imshow(clahe_image, cmap="gray") # Display CLAHE processed image
    axes[1].set_title("Clahe processed image")
    axes[1].axis("off")
    axes[2].imshow(otsu_mask, cmap="gray") # Display Otsu's threshold mask
    axes[2].set_title("Otsu’s Threshold Mask")
    axes[2].axis("off")
    plt.show() # Show the figure
    cv2.imwrite(output_mip_path, gray_image) # Save MIP image
    cv2.imwrite(output_mask_path, otsu_mask) # Save Otsu mask image
    print(f"Images saved: {output_mip_path} and {output_mask_path}")

def visualize_segmented_regions(gray_image, seg_im, cmap_name='hsv', min_vessel_size = min_vessel_size):
    """
    Visualise labelled vessel segments overlaid on the greyscale image.
    
    Args:
        gray_image (ndarray): Greyscale MIP image for background.
        seg_im (ndarray): Binary vessel segmentation mask.
        cmap_name (str): Matplotlib colormap name for segment colouring.
        min_vessel_size (int): Minimum segment size to keep (pixels).

    Returns:
        labeled_segments (ndarray): Labelled segmentation image with unique integer labels for each segment.
        Returns None on failure.
    """
    try:
        gray_image_norm = (gray_image - gray_image.min()) / (gray_image.max() - gray_image.min()) # Normalise grayscale image for better visualisation
        labeled_segments = label(seg_im) # Label each vessel segment
        num_labels = np.max(labeled_segments) # Get number of unique segments
        labeled_segments = remove_small_objects(labeled_segments, min_size=min_vessel_size) # Remove small vessel segments based on minimum vessel size
        if num_labels == 0: # Check if any segments are detected
            print("No segments detected.")
            return None
        cmap = matplotlib.colormaps.get_cmap(cmap_name) # Generate colours from the chosen colourmap
        colors = cmap(np.linspace(0, 1, num_labels))[:, :3] # Get RGB values for each label
        overlay = label2rgb(labeled_segments, image=gray_image_norm, bg_label=0, alpha=0.6, colors=colors) # Overlay coloured segments on grayscale image
        # Create a figure
        fig, ax = plt.subplots(figsize=(8, 6)) # Set figure size
        ax.imshow(overlay, cmap='gray') # Display overlay image
        ax.set_title("Vessel Segmentation Visualization")
        for region in regionprops(labeled_segments): # Loop through each labeled region
            centroid = region.centroid # Plot segment labels at centroids
            ax.text(centroid[1], centroid[0], str(region.label), color='white', fontsize=8, ha='center', va='center', fontweight='bold')
        ax.axis('off') # Remove axes
        plt.show() # Show the figure
        return labeled_segments  # Return labeled segments for further processing
    except Exception as e: 
        print(f"Error visualizing segmented regions: {e}") # Print error message if visualization fails
        return None
    
def remove_selected_segments(labeled_segments, remove_list):
    """
    Manual curation: Remove user-selected labelled segments from a segmentation mask.

    Args:
        labeled_segments (ndarray): Labelled segmentation image.
        remove_list (list[int] or None): Segment labels to remove. Use the segment numbers displayed in the visualisation for reference. If None, no segments will be removed.

    Returns:
        cleaned_mask (ndarray): Binary mask with curated segments 
        Returns None on failure.
    """
    try:
        mask = np.isin(labeled_segments, remove_list, invert=True) # Create mask to exclude selected segments
        cleaned_mask = labeled_segments * mask # Apply mask to labeled segments
        return cleaned_mask > 0 # Return manually curated binary mask
    except Exception as e:
        print(f"Error removing selected segments: {e}") # Print error message if removal fails
        return None

def segment_and_analyze_vessels(file_name, image, gray_image, clahe_image, otsu_mask, output_path, 
                                im_filter=seg_method, sigma1=sigma1, hole_size=hole_size, 
                                ditzle_size=ditzle_size, thresh=thresh):
    """
    Vessel segmentation, optional apply of manual curation and results visualisation
    
    Args:
        file_name (str): File name used to derive the sample ID.
        image (ndarray): Raw image data used for segmentation.
        gray_image (ndarray): Greyscale MIP image.
        clahe_image (ndarray): CLAHE-enhanced greyscale image.
        otsu_mask (ndarray): Binary mask from Otsu thresholding.
        output_path (str): Directory for saving figures.
        im_filter (str): Vessel enhancement filter name (meijering, frangi, sato or jerman).
        sigma1 (range): Sigma range for vessel enhancement.
        hole_size (int): Max hole size to fill in the mask.
        ditzle_size (int): Max size of small objects to remove.
        thresh (int): Threshold for binarisation of the enhanced image.

    Returns:
        tuple:
            vessel_seg (ndarray): Final vessel segmentation mask (curated if manual curation is applied).
            remove_list (list[int] or None): Labels removed by the user.
        Returns None on failure.
    """
    try:
        # Segment vessels using specified filtering method and parameters
        vessel_seg = vm.segment_image(image,
                                      im_filter=im_filter, # Vessel segmentation method
                                      sigma1=sigma1, # Sigma range for vessel enhancement filter
                                      hole_size=hole_size, # Hole size to be filled in the vessel mask
                                      ditzle_size=ditzle_size, # Size of small objects to be removed from the vessel mask
                                      thresh=thresh) # Threshold value to binarise the vessel enhanced image
        seg_im = vessel_seg.astype(np.uint8) # Convert binary mask to uint8 format
        # Visualise initial segmentation results
        fig, axes = plt.subplots(1, 4, figsize=(15, 5)) # Create figure with 4 panels
        axes[0].imshow(gray_image, cmap="gray") # Display MIP
        axes[0].set_title("Maximum Intensity Projection (MIP)")
        axes[0].axis("off")
        axes[1].imshow(clahe_image, cmap="gray") # Display CLAHE processed image
        axes[1].set_title("Clahe processed image")
        axes[1].axis("off")
        axes[2].imshow(seg_im, cmap="gray") # Display vessel segmentation image
        axes[2].set_title("Vessel segmentation image")
        axes[2].axis("off")
        axes[3].imshow(otsu_mask, cmap="gray") # Display Otsu's threshold mask
        axes[3].set_title("Otsu’s Threshold Mask")
        axes[3].axis("off")
        plt.show() # Show the figure
        sampleID = file_name.split('.tiff')[0] 
        fig_title = f"{sampleID}_segmentation_mask_prefiltering" # Title for the figure
        output_pdf_path = os.path.join(output_path, f"{fig_title}.pdf") # Define output PDF path
        save_plot_as_pdf(fig, output_pdf_path) # Save figure as PDF
        # Visualise segmented regions with labels for manual curation    
        labeled_segments = visualize_segmented_regions(gray_image, seg_im, cmap_name='hsv', min_vessel_size = min_vessel_size) # Visualise segmented regions
        sampleID = file_name.split('.tiff')[0]
        fig_title = f"{sampleID}_segmentation_labelled" # Title for the figure
        output_pdf_path = os.path.join(output_path, f"{fig_title}.pdf") # Define output PDF path
        save_plot_as_pdf(fig, output_pdf_path) # Save figure as PDF
        # Manual curation: Remove unwanted segments based on user input
        remove_list = input("Enter segment numbers to remove (comma-separated) or press Enter to skip: ").strip() # Get user input for segments to remove
        if remove_list:
            remove_list = [int(x) for x in remove_list.split(",")] # Convert input string to list of integers
            cleaned_mask = remove_selected_segments(labeled_segments, remove_list) # Remove selected segments
            labeled_segments = label(cleaned_mask) # Relabel the manually curated mask
            vessel_seg = labeled_segments # Update vessel segmentation mask
        else:
            remove_list = None # No segments removed
            cleaned_mask = remove_selected_segments(labeled_segments, remove_list) # Keep original mask
            labeled_segments = label(cleaned_mask) 
            vessel_seg = labeled_segments
        # Visualise manually curated segmentation results    
        print(f"Visualising filtered segmentation image")
        fig, axes = plt.subplots(1, 4, figsize=(15, 5)) # Create figure with 4 panels
        axes[0].imshow(gray_image, cmap="gray")
        axes[0].set_title("Maximum Intensity Projection (MIP)") # Display MIP
        axes[0].axis("off")
        axes[1].imshow(clahe_image, cmap="gray") # Display CLAHE processed image
        axes[1].set_title("Clahe processed image")
        axes[1].axis("off")
        axes[2].imshow(cleaned_mask, cmap="gray") # Display manually filtered vessel segmentation image
        axes[2].set_title("Filtered vessel segmentation image")
        axes[2].axis("off")
        axes[3].imshow(otsu_mask, cmap="gray") # Display Otsu's threshold mask
        axes[3].set_title("Otsu’s Threshold Mask")
        axes[3].axis("off")
        plt.show() # Show the figure
        sampleID = file_name.split('.tiff')[0]
        fig_title = f"{sampleID}_segmentation_mask" # Title for the figure
        output_pdf_path = os.path.join(output_path, f"{fig_title}.pdf") # Define output PDF path
        save_plot_as_pdf(fig, output_pdf_path) # Save figure as PDF  
        return vessel_seg, remove_list
    except Exception as e:
        print(f"Error segmenting vessels for {file_name}: {e}") # Print error message if segmentation fails
        return None

def calculate_and_print_metrics(label_path, seg_mask):
    """
    Compute vessel metrics and overlap statistics.

    Args:
        label_path (str): Path to the labelled image.
        vessel_seg (ndarray): Vessel segmentation mask.

    Returns:
        tuple:
            length (float): Vessel length metric.
            area (float): Vessel area metric.
            conn (float): Connectivity metric.
            Q (float): Combined metric of Q = connectivity * length * area.
            jaccard (float): Jaccard index between label and segmentation.
            label (ndarray): Raw labelled image.
            label_im (ndarray): Labelled image cast to uint8.
        Returns None on failure.
    """
    try:
        label = cv2.imread(label_path, 0) # Load label image
        label_im = label.astype(np.uint8) # Convert label image to uint8
        seg_im = seg_mask[i].astype(np.uint8) # Convert segmentation mask to uint8, [i] is the index of each ROI
        length, area, conn, Q = vm.cal(label_im, seg_im) # Calculate vessel metrics: length, area, connectivity and Q factor (combined metric of connectivity * length * area)
        jaccard = vm.jaccard(label_im, seg_im) # Calculate Jaccard index between label and segmentation mask
        return length, area, conn, Q, jaccard, label, label_im # Return vessel metrics and labelled images
    except Exception as e:
        print(f"Error calculating metrics: {e}") # Print error message if vessel metrics calculation fails
        return None

def find_branchpoints(skel, min_distance=Branch_dist):
    """
    Detect branchpoints from a skeleton image and removes nearby branchpoints based on a minimum distance threshold.

    Args:
        skel (ndarray): Skeletonised vessel image.
        min_distance (int): Minimum distance between branchpoints (pixels). Users are reccommended to set this parameter based median(vessel_length)

    Returns:
        tuple:
            edges (ndarray): Skeleton edges after removing branchpoints.
            branchpoints (ndarray): Binary image of cleaned branchpoints.
    """
    skel_binary = np.zeros_like(skel) # Create binary skeleton image
    skel_binary[skel > 0] = 1 # Binarise skeleton image
    skel_index = np.argwhere(skel_binary == True) # Get coordinates of skeleton pixels
    tile_sum = [] # List to store sum of neighbourhood pixels
    neighborhood_image = np.zeros(skel.shape) # Image to store neighbourhood sums
    for i, j in skel_index: # Loop through each skeleton pixel
        this_tile = skel_binary[i - 1 : i + 2, j - 1 : j + 2] # Extract 3x3 neighbourhood
        tile_sum.append(np.sum(this_tile)) # Sum of neighbourhood pixels
        neighborhood_image[i, j] = np.sum(this_tile) # Store sum in neighbourhood image    
    branch_points = np.zeros_like(neighborhood_image) # Create branchpoint image
    branch_points[neighborhood_image > 3] = 1 # Identify branchpoints (neighbourhood sum > 3)
    branch_points = branch_points.astype(np.uint8) # Convert branchpoint image to uint8
    branch_coords = np.argwhere(branch_points == 1) # Get coordinates of branchpoints
    to_keep = np.ones(len(branch_coords), dtype=bool) # Boolean array to track branchpoints to keep
    dists = distance.squareform(distance.pdist(branch_coords)) # Calculate pairwise distances between branchpoints
    for i in range(len(branch_coords)): # Loop through each branchpoint
        if not to_keep[i]: # Skip if already marked for removal
            continue
        close = np.where((dists[i] < min_distance) & (dists[i] > 0))[0] # Find nearby branchpoints within min_distance
        to_keep[close] = False  # Keep only the first point and remove closeby branchpoints
    cleaned_branch_points = np.zeros_like(skel, dtype=np.uint8) # Creates empty image to store filtered branchpoints
    for idx in np.where(to_keep)[0]: # Loop through the index of kept branchpoints
        y, x = branch_coords[idx] # Get coordinates for each branchpoint
        cleaned_branch_points[y, x] = 1 # Mark the location of the branchpoint
    edges = skel_binary - cleaned_branch_points # Edges are defined as skeleton pixels minus the cleaned branchpoints
    edges[edges < 0] = 0 # Ensures no negative values are stored
    branchpoints = cleaned_branch_points # Store filtered branchpoints
    return edges.astype(np.uint8), branchpoints

def analyze_skeleton(seg_mask, gray_image, file_name):
    """
    Analyse skeleton and vessel diameter. Visualise skeleton and branchpoints and save figures as PDF.
    
    Args:
        vessel_seg (ndarray): Vessel segmentation mask.
        gray_image (ndarray): Greyscale MIP image.
        file_name (str): File name used to derive the sample ID.

    Returns:
        tuple:
            skel (ndarray): Skeletonised image.
            edges (ndarray): Skeleton edges with branchpoints removed.
            edge_labels (ndarray): Labelled edges image.
            branchpoints (ndarray): Binary branchpoint image.
            coords (list): Edge coordinates from endpoint analysis.
            endpoints (list): Endpoint coordinates from endpoint analysis.
        Returns None on failure.
    """
    try:
        skel, edges, _ = vm.skeletonize_vm(seg_mask[i], method=skel_method) # Calculate skeleton metrics and edges of each image, [i] is the index of each ROI
        _, edge_labels = cv2.connectedComponents(edges) # Label connected edges with a unique identifier
        _, branchpoints = find_branchpoints(skel) # Define branhpoints (refined from find_branchpoints)
        coords, endpoints = vm.find_endpoints(edges) # Find coordinates of each edge for further analysis and visualisation
        # Visualise skeleton plot
        cmap1 = matplotlib.colors.LinearSegmentedColormap.from_list("", [(1, 1, 1, 0), (0, 0, 0, 1)]) # Colourmap for skeleton (transparent to black)
        cmap2 = matplotlib.colors.LinearSegmentedColormap.from_list("", [(1, 1, 1, 0), (1, 0, 0, 1)]) # Colourmap for overlay (transparent to red)
        fig, axes = plt.subplots(1, 3, figsize=(15, 5)) # Create a figure with 3 panels
        axes[0].imshow(gray_image, cmap="gray") # Display MIP
        axes[0].set_title("Maximum Intensity Projection (MIP)") 
        axes[0].axis("off")
        axes[1].imshow(skel, cmap=cmap1) # Display skeleton
        axes[1].set_title("Skeleton Plot") 
        axes[1].axis("off")
        axes[2].imshow(gray_image, cmap="gray") # Display overlay of skeleton over MIP
        axes[2].imshow(skel, cmap=cmap2)
        axes[2].set_title("Overlap")
        axes[2].axis("off")
        plt.show() # Show the figure
        sampleID = file_name.split('.tiff')[0]
        fig_title = f"{sampleID}_skeleton_plot" # Title for the figure
        output_pdf_path = os.path.join(output_path, f"{fig_title}.pdf") # Define output PDF path
        save_plot_as_pdf(fig, output_pdf_path) # Save figure as PDF
        # Visualise skeleton lines   
        kernel = np.ones((10, 10), np.uint8)  # Adjust kernel size to make branchpoints bigger 
        branchpoints_dilated = cv2.dilate(branchpoints.astype(np.uint8), kernel, iterations=1) # Dilate branchpoints to make them more visible on the image (only for visualisation purposes)
        fig, axes = plt.subplots(1, 3, figsize=(15, 5)) # Create a figure with 3 panels
        axes[0].imshow(gray_image, cmap="gray") # Display MIP
        axes[0].set_title("Maximum Intensity Projection (MIP)")
        axes[0].axis("off")
        axes[1].imshow(branchpoints_dilated, cmap='gray_r') # Display branchpoints
        axes[1].set_title("Branchpoints")
        axes[1].axis("off")
        axes[2].imshow(gray_image, cmap="gray") # Display overlay of branchpoints over MIP
        axes[2].imshow(branchpoints_dilated, cmap=cmap2)
        axes[2].set_title("Overlap")
        axes[2].axis("off")
        plt.show() # Show the figure
        sampleID = file_name.split('.tiff')[0]
        fig_title = f"{sampleID}_branchpoint_plot" # Title for the figure
        output_pdf_path = os.path.join(output_path, f"{fig_title}.pdf") # Define output PDF path
        save_plot_as_pdf(fig, output_pdf_path) # Save figure as PDF
        return skel, edges, edge_labels, branchpoints, coords, endpoints
    except Exception as e:
        print(f"Error analyzing skeleton: {e}") # Print error message if skeletonisation fails
        return None

def is_outlier(series, threshold_low=threshold_low, threshold_high=threshold_high):
    """
    Identify outliers using Median Absolute Deviation (MAD).

    Args:
        series (pandas.Series): Series of numeric values to calculate central tendency.
        threshold_low (float): Lower MAD threshold.
        threshold_high (float): Upper MAD threshold.

    Returns:
        pandas.Series: Boolean mask where True indicates an outlier.
    """
    median = series.median() # Calculates central tendency using median
    mad = median_abs_deviation(series) # Calculates non-scaled MAD dispersion
    # Compute assymetric bounds using provided thresholds
    lower_bound = median - threshold_low * mad # Lower threshold
    upper_bound = median + threshold_high * mad # Upper threshold
    return (series < lower_bound) | (series > upper_bound) # Return a boolean with outliers

def DF_filtering(df):
    """
    Filter outliers from each column of a DataFrame using MAD.

    Args:
        df (pandas.DataFrame): DataFrame to filter column-wise based on MAD outlier analysis.

    Returns:
        pandas.DataFrame: DataFrame with outliers removed per column.
    """
    filtered_dict = {} # Empty dictionary to store filtered dataframes
    for column in df.columns: # Iterate through each column 
        data = df[column].dropna()  # Removes missing values to avoid propagating NaNs into MAD and comparisons
        outliers = is_outlier(data)  # Compute outlier mask for the current column
        filtered_data = data[~outliers]  # Keep only non-outlier values
        filtered_dict[column] = filtered_data.reset_index(drop=True) # Reset index and store filtered data in the dictionary
    filtered_values_df = pd.DataFrame(filtered_dict) # Build new dataframe with filtered results
    return filtered_values_df

def find_crossline_length(bx, by, point, im, d = dist, diam = diam):
    """
    Estimate crossline length across a vessel segment. Iteratively expands the crossline distance until two edge transitions are found or bounds are exceeded.
    
    Args:
        bx (float): X component of the crossline slope.
        by (float): Y component of the crossline slope.
        point (ndarray): Coordinates of the point on the vessel segment where the crossline is centered.
        im (ndarray): Binary image of the vessel segmentation mask.
        d (int): Initial distance for crossline search (pixels).
        diam (int): Initial diameter value (pixels).
    
    Returns:
        length (float): Estimated crossline length converted to microns.
    """
    d = d  # Define small starting distance
    diam = diam  # Define small starting diameter
    im_h, im_w = im.shape # Define image bounds
    max_d = min(im_h, im_w) // 4  # Cap the analysis distance to avoid scanning beyond ROI size
    while diam == diam:
        d += 1  #  Increase the crossline search span in small increments (finer steps for smaller vessels)
        _, cross_index = vm.make_crossline(bx, by, point, d) # Returns pixel indices of the crossline coordinates
        if all(0 <= i[0] < im_h and 0 <= i[1] < im_w for i in cross_index): # Check that all candidate indices lie within image bounds
            seg_val = [im[i[0], i[1]] for i in cross_index] # Sample binary images along crosslines
            steps = np.where(np.roll(seg_val, 1) != seg_val)[0] # Find positions where value changes (edges)
            if len(steps) >= 2:
                diam = abs(steps[-1] - steps[0]) # Use distance between first and last transition as a proxy to calculate diameter (in pixels)
            if d > max_d: 
                break # Stop if search span exceeds image bounds
        else:
            break # Break if the crossline stepped outside the image
    length = diam * scale if diam > diam else 0 # Scales lenght measurements using diameter values (pixels) and empirical scale (from the microscope settings)
    return length

def visualize_vessel_diameter(edge_labels, segment_number, seg, im, use_label=False, pad=True, n_points=points):
    """
    Estimate vessel diameters along a single segment and visualise crosslines.
    
    Args:
        edge_labels (ndarray): Labelled edges image where each vessel segment has a unique integer label.
        segment_number (int): The specific segment label to analyse.
        seg (ndarray): Binary vessel segmentation mask.
        im (ndarray): Greyscale MIP image for diameter measurement.
        use_label (bool): Whether to use the binary segmentation mask (True) or the raw image (False) for diameter estimation.
        pad (bool): Whether to apply padding to the images to avoid edge artifacts when measuring near borders.
        n_points (int): Number of points to sample along the vessel segment for diameter measurement.
    
    Returns:
        tuple:
            diameter (list): List of diameter measurements across the vessel segment.
            mean_diameter (float): Mean diameter of the vessel segment.
            viz (ndarray): Image visualising the crosslines used for diameter measurement.
    """
    # Optional padding to avoid edge artifacts when measuring crosslines near ROI borders
    if pad == True: 
        pad_size = pad_size
        edge_labels = np.pad(edge_labels, pad_size)
        seg = np.pad(seg, pad_size)
        im = np.pad(im, pad_size)
    segment = np.zeros_like(edge_labels) # Extract the binary mask of selected labels
    segment[edge_labels == segment_number] = 1 
    segment_median = vm.segment_midpoint(segment) # Choose a representative point on the vessel segment (e.g. median of indices)
    vx, vy = vm.tangent_slope(segment, segment_median) # Calculate local tangent vector
    bx, by = vm.crossline_slope(vx, vy) # Calculate perpendicular crossline of the tangent vector
    viz = np.zeros_like(seg) # Define crossline footprints for visualisation
    cross_length = find_crossline_length(bx, by, segment_median, seg) # Determine the crossline length to use across the vessel
    if cross_length == 0: # If no available crossline length obtained
        diameter = 0 # Diameter equals to zero
        mean_diameter = 0 # Mean diameter of the vessel segment equals to zero
        print("cross length 0") # Print warning message defining crossline length value as zero
        return diameter, mean_diameter, viz
    diameter = [] # Define empty list to store diameter measurements across the labelled vessel segment
    segment_inds = np.argwhere(segment) # Retrieve the indices where the segment is present
    num_points = min(n_points, len(segment_inds)) # Ensure never to exceed number of available points
    sample_indices = np.linspace(0, len(segment_inds)-1, num_points, dtype=int) # Creates a range with defined number of steps
    for i in sample_indices:  # Sample every defined num_points (lower values will require more computation power and processing time)
        this_point = segment_inds[i] # Retrieves each point
        vx, vy = vm.tangent_slope(segment, this_point) # Re-calculate local tangent vector
        bx, by = vm.crossline_slope(vx, vy) # Re-calculate perpendicular crossline of the tangent vector
        _, cross_index = vm.make_crossline(bx, by, this_point, cross_length) # Build crossline indices with the chosen length
        if use_label: # Call crosslines from the labelled segmentation
            cross_vals = vm.crossline_intensity(cross_index, seg) # Extract intensity values along the crossline from the vessel segment
            diam = vm.label_diameter(cross_vals) # Calculate diameter based on labelled object width
        else: # Call crosslines from raw image
            cross_vals = vm.crossline_intensity(cross_index, im) # Extract intensity values along the crossline
            diam = vm.fwhm_diameter(cross_vals) # Calculate diameter using full width at half maximum (FWHM)
        # For visualisation only
        val = 5 if diam == 0 else 10 # Generate a short crossline if no diameter was found, otherwise plot crossline
        for ind in cross_index:
            viz[ind[0], ind[1]] = val
        diameter.append(diam) # Store diameters
    diameter = [x for x in diameter if x != 0] # Remove zero entries (diameters not identified)
    if diameter:
        mean_diameter = np.mean(diameter) # Compute mean of the diameters if valid measurements exist
    else:
        mean_diameter = 0
    if pad == True: #  If padded earlier, crop visualisation back to original shape
        im_shape = edge_labels.shape
        viz = viz[
            pad_size : im_shape[0] - pad_size,
            pad_size : im_shape[1] - pad_size,
        ]
    return diameter, mean_diameter, viz

def whole_anatomy_diameter(im, seg, edge_labels, minimum_length=minimum_length, pad_size=pad_size):
    """
    Measure vessel diameters for all segments above a length threshold in the defined ROI and visualise the crosslines used for diameter measurement.
    
    Args:
        im (ndarray): Greyscale MIP image for diameter measurement.
        seg (ndarray): Binary vessel segmentation mask.
        edge_labels (ndarray): Labelled edges image where each vessel segment has a unique integer label.
        minimum_length (int): Minimum length threshold for vessel segments to be included in diameter analysis (pixels).
        pad_size (int): Size of padding to apply to images to avoid edge artifacts when measuring diameters near borders (pixels).
    
    Returns:
        tuple:
            full_viz_no_pad (ndarray): Image visualising the crosslines used for diameter measurement across the entire ROI.
            output_diameters (list): List of tuples pairing each included vessel segment with its corresponding list of diameter measurements.
    """
    unique_edges = np.unique(edge_labels) # Retrieve labels for each unique vessel segment
    unique_edges = np.delete(unique_edges, 0) # Remove background segments
    # Optional padding to avoid edge artifacts when measuring diameters near ROI borders
    edge_label_pad = np.pad(edge_labels, pad_size)
    seg_pad = np.pad(seg, pad_size)
    im_pad = np.pad(im, pad_size)
    full_viz = np.zeros_like(seg_pad) # Creates a visualisation layer
    diameters = [] # Empty list to store diameter
    included_segments = [] # Empty list to store vessel segments
    for i in unique_edges: # Loop through each vessel segment
        seg_length = len(np.argwhere(edge_label_pad == i)) # Calculate the length of each segment (in pixels)
        if seg_length > minimum_length: # Only analyse segments above a minimum length threshold
            included_segments.append(i) # Save each vessel segment
            _, temp_diam, temp_viz = visualize_vessel_diameter(edge_label_pad, i, seg_pad, im_pad, pad=False, n_points=points) # Calculate vessel diameters and visualise them
            diameters.append(temp_diam) # Save each vessel diameter
            full_viz = full_viz + temp_viz # Accumulate visualisation layers
    # Remove earlier padding before returning final output
    im_shape = edge_label_pad.shape
    full_viz_no_pad = full_viz[pad_size : im_shape[0] - pad_size, pad_size : im_shape[1] - pad_size]
    output_diameters = list(zip(included_segments, diameters)) # Pair each vessel segment with its corresponding diameter list
    return full_viz_no_pad, output_diameters

def analyze_vessel_diameters(mip_imageROI, seg_mask, edge_labels, gray_image, file_name):
    """
    Compute vessel diameters and visualise diameter crosslines.

    Args:
        mip_imageROI (ndarray): Raw image data used for diameter extraction.
        seg_mask (ndarray): Vessel segmentation mask.
        edge_labels (ndarray): Labelled edges image from skeletonisation.
        gray_image (ndarray): Greyscale MIP image.
        file_name (str): File name used to derive the sample ID.

    Returns:
        tuple:
            diam_values (list[float]): Diameter values in pixels.
            viz (ndarray): Visualisation image of diameter crosslines.
            filtered_diam_values (pandas.DataFrame): MAD-filtered diameters.
        Returns ([], None, None) on failure.
    """
    try:
        viz, diameters = whole_anatomy_diameter(mip_imageROI, seg_mask, edge_labels) # Extract diameter measurements from the entire ROI
        cmap2 = matplotlib.colors.LinearSegmentedColormap.from_list("", [(1, 1, 1, 0), (1, 0, 0, 1)]) # Define colourmap for overlay (transparent to red)
        fig, axes = plt.subplots(1, 3, figsize=(15, 5)) # Figure with 3 panels
        axes[0].imshow(gray_image, cmap="gray") # Display MIP
        axes[0].set_title("Maximum Intensity Projection (MIP)")
        axes[0].axis("off")
        axes[1].imshow(viz, cmap="gray_r") # Display diameter crosslines
        axes[1].set_title("Diameters")
        axes[1].axis("off")
        axes[2].imshow(gray_image, cmap="gray") # Display overlay of diameter crosslines over MIP
        axes[2].imshow(viz, cmap=cmap2)
        axes[2].set_title("Overlap")
        axes[2].axis("off")
        plt.show() # Show the figure
        sampleID = file_name.split('.tiff')[0]
        fig_title = f"{sampleID}_diameter_plot" # Title for the figure
        output_pdf_path = os.path.join(output_path, f"{fig_title}.pdf") # Define output PDF path
        save_plot_as_pdf(fig, output_pdf_path) # Save figure as PDF
        num_diameters = diameters[-1][0] # Re-index diameters with unique identifiers after previous diameter removal
        indexed_diameters = [] # Empty list to store new indexed diameters
        for new_index in range(1, num_diameters + 1): # Loop through each diameter identifier
            found = next((d for d in diameters if d[0] == new_index), None) # Find unique identifiers
            if found:
                indexed_diameters.append((new_index, found[1])) # If vessel ID was found, keep identifier
        filtered_diameters = [(idx, diam) for idx, diam in indexed_diameters if diam > 0] # Remove crosslines that present diameters with zero values
        diam_values = [diam for idx, diam in filtered_diameters] # Extract only positive diameter values        
        filtered_diam_values = DF_filtering(pd.DataFrame(diam_values, columns=['Diameter (px)'])) # MAD-based filtering of outliers
        return diam_values, viz, filtered_diam_values  # Return the diameter values
    except Exception as e:
        print(f"Error analyzing vessel diameters: {e}") # Print error message if diameter analysis fails
        return [], None, None  # Return an empty list in case of an error

def visualize_vessel_metrics(edge_labels, diam_values, scale = scale):
    """
    Calculate vessel length, tortuosity, and diameter metrics and performs MAD filtering.

    Args:
        edge_labels (ndarray): Labelled edges image from skeletonisation.
        diam_values (list[float]): Vessel diameter values in pixels.
        scale (float): Pixels per micron for spatial conversion. Users need to set this parameter based on the imaging resolution of their dataset.

    Returns:
        tuple:
            length (ndarray): Segment lengths in pixels.
            vessel_len (float): Mean segment length in pixels.
            tort (ndarray): Tortuosity values per segment.
            vessel_tor (float): Mean tortuosity.
            diam_values_um (ndarray): Diameters converted to microns.
            length_um (ndarray): Segment lengths converted to microns.
            filtered_length_df (pandas.DataFrame): MAD-filtered lengths (pixels).
            filtered_length_um_df (pandas.DataFrame): MAD-filtered lengths (microns).
            filtered_diam_values_df (pandas.DataFrame): MAD-filtered diameters (pixels).
            filtered_diam_values_um_df (pandas.DataFrame): MAD-filtered diameters (microns).
        Returns None on failure.
    """
    try:
        _, length = vm.vessel_length(edge_labels) # Calculate the length of each segment (pixels)
        vessel_len = np.mean(length) # Compute the mean of segment lengths
        tort, _ = vm.tortuosity(edge_labels) # Calculate tortuosity values
        vessel_tor = np.mean(tort) # Compute the mean tortuosity
        tort = np.array(tort) # Transform values into array
        # Spatial unit convertion: pixels to microns
        diam_values_um = np.array(diam_values) / scale # Convert diameter values
        length_um = length / scale # Convert segment length values
        filtered_length_df = DF_filtering(pd.DataFrame(length)) # Filter outliers for segment length in pixels
        filtered_length_um_df = DF_filtering(pd.DataFrame(length_um)) # Filter outliers for segment length in microns
        filtered_diam_values_df = DF_filtering(pd.DataFrame(diam_values)) # Filter outliers for vessel diameters in pixels
        filtered_diam_values_um_df = DF_filtering(pd.DataFrame(diam_values_um)) # Filter outliers for vessel diameters in microns
        return length, vessel_len, tort, vessel_tor, diam_values_um, length_um, filtered_length_df, filtered_length_um_df, filtered_diam_values_df, filtered_diam_values_um_df
    except Exception as e:
        print(f"Error visualizing vessel metrics: {e}") # Print error message if vessel measurement fails
        return None

def analyze_vessel_network(mip_imageROI, seg_mask, edges, label, scale = scale, window = window_size):
    """
    Compute network-level metrics such as total length and density.

    Args:
        mip_imageROI (ndarray): Raw image data.
        seg_mask (ndarray): Vessel segmentation mask.
        edges (ndarray): Skeleton edges image.
        label (ndarray): Labelled image for branchpoint density.
        scale (float): Pixels per micron for spatial conversion.
        window (int): Window size (microns) for density binning.

    Returns:
        tuple:
            net_length (float): Total network length in pixels.
            net_length_um (float): Total network length in microns.
            density (float): Mean vessel density.
            density_array (ndarray): Bin density values.
            overlay (ndarray): Density overlay image.
            vessel_density (float): Fraction of vessel in micron².
            bp_density (tuple): Branchpoint density outputs.
        Returns None on failure.
    """
    density_scale = np.floor(scale * window).astype(int) # Convert pixel scale to a 10×10 grid corresponding to 100µm² bins
    try:
        net_length = vm.network_length(edges) # Calculate total network length (sum of vessel segments)
        net_length_um = net_length / scale # Convert total network length into microns
        density, density_array, overlay = vm.vessel_density(mip_imageROI, seg_mask, density_scale, density_scale) # Compute vessel density map
        vessel_density = sum(density_array) # Number of vessel pixels versus total pixels
        bp_density = vm.branchpoint_density(label) # Optional: Calculates branchpoint density
        return net_length, net_length_um, density, density_array, overlay, vessel_density, bp_density
    except Exception as e:
        print(f"Error analyzing vessel network: {e}") # Print error message if network measurements fail

def save_roi_results_to_excel(roi_results, roi_raw, output_path=None, output_name=output_name):
    """
    Save ROI vessel analysis metrics and raw values to an Excel file.

    Args:
        results (list[dict]): Summary metrics per ROI (one dict per ROI).
        roi_raw (list[dict]): Raw measurement values per ROI (one dict per ROI).
        output_path (str): Directory path to save the Excel file. If None, saves to current working directory.
    Returns:
        None
    """
    try:
        df_roi_metrics = pd.DataFrame(roi_results) # Converts summary results to dataframe
        roi_raw_dict = {} # Empty dictionary to collect ROI raw data for Excel output
        for col in roi_raw[0].keys(): # Loop through each column
            if col == "Sample ID": # Check ROI identifier
                continue
            sample_dict = {} # Empty dictionary to collect sample-specific data for Excel output
            for roi_dict in roi_raw: # Loop through each column of the raw dataframe 
                sampleID = roi_dict["Sample ID"] # Saves ROI identifier
                values = roi_dict[col] # Saves data
                # Flatten arrays and convert to list format
                if isinstance(values, np.ndarray):
                    values = values.flatten()
                elif not isinstance(values, list):
                    values = list(values)
                sample_dict[sampleID] = values # Store metrics
            roi_raw_dict[col] = sample_dict # Save the metrics for each ROI in the original dictionary
        # Build Excel file
        output_excel_path = os.path.join(output_path, f"{output_name}.xlsx") # Defines path to output Excel file
        with pd.ExcelWriter(output_excel_path) as writer:
            # Sheet 1: summary of ROI metrics
            df_roi_metrics.to_excel(writer, sheet_name="ROI Metrics", index=False)
            # Additional sheets: ROI raw measurements
            for measurement, sample_dict in roi_raw_dict.items():
                df = pd.DataFrame({k: pd.Series(v) for k, v in sample_dict.items()})
                df = df.apply(lambda col: col.dropna().astype(int).reset_index(drop=True)) # Remove NaN per column and convert to int
                sheet_name = measurement[:31]  # Excel sheet name limit
                df.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"ROI results saved to {output_excel_path}.")
    except Exception as e:
        print(f"Error saving ROI results to Excel: {e}")  # Print error message if writter for Excel file fails

## <a id='toc3_3_'></a>[Define ROI, perform vessel segmentation and mask filtering](#toc0_)

### <a id='toc3_3_1_'></a>[For single channel](#toc0_)

#### <a id='toc3_3_1_1_'></a>[Run samples in working directory](#toc0_)

In [ ]:
# Define file name and extension. Collect all files in the directory that match the name prefix
file_names = [f for f in os.listdir(data_path) if f.startswith(name_start) and f.endswith(".czi")]

for file_name in file_names: # Loop through each file in the working directory
    file_path = os.path.join(data_path, file_name) # Build full path to the file
    sampleID = file_name.split('_MaxInt')[0] # Extract sample identifier 
    print(f"\n--- Processing {sampleID} ---")
    (image,                 # Original mage
     roi_masks,             # ROI mask
     roi_coords,            # ROI coordinates
     mip_image,             # Maximum intensity projection (MIP)
     gray_image,            # Greyscale MIP
     clahe_image,           # CLAHE-enhanced MIP
     otsu_mask,             # OTSU threshold mask
     output_mip_path,       # Output path for MIP
     output_mask_path,      # Output path for mask
     mip_imageROI,          # List of ROI MIP images
     roi_grayROI,           # List of ROI grayscale images
     roi_claheROI,          # List of ROI CLAHE images
     roi_otsuROI,           # List of ROI OTSU masks
     output_mipROI_paths,   # Output paths for ROI MIPs
     output_maskROI_paths,  # Output paths for ROI masks
    ) = load_and_process_czi_singleROI(file_path, file_name, output_path, roi = ROI_num) # Load and process the image (single channel)
    # Store all ROI-level outputs for later analysis
    all_mipROIs.extend(mip_imageROI)
    all_grayROIs.extend(roi_grayROI)
    all_claheROIs.extend(roi_claheROI)
    all_otsuROIs.extend(roi_otsuROI)
    output_ROI_paths.extend(output_mipROI_paths)
    output_ROImask_paths.extend(output_maskROI_paths)
    all_coords.extend(roi_coords)
    if mip_imageROI is None:
            continue      # Skip this file if ROI extraction failed

#### <a id='toc3_3_1_2_'></a>[Generate vessel segmentation masks](#toc0_)

In [ ]:
# Note: The user is reccommended to clear all jupyter outputs to save memory
for i in range(len(all_mipROIs)): # Loop through each ROI to perform vessel segmentation
    # Extract sample ID and filename from the ROI output path
    sampleID = output_ROI_paths[i].split("/")[-1].split('.tiff')[0]
    sample_name = output_ROI_paths[i].split("/")[-1]
    # Vessel segmentation within the defined ROI
    vessel_seg, remove_list = segment_and_analyze_vessels(file_name=sample_name,        # Sample identifier
                                                          image=all_mipROIs[i],         # MIP ROI
                                                          gray_image=all_grayROIs[i],   # Greyscale ROI
                                                          clahe_image=all_claheROIs[i], # CLAHE-enhanced ROI
                                                          otsu_mask=all_otsuROIs[i],    # OTSU threshold ROI
                                                          output_path=output_path,      # Path to output directory
                                                          im_filter=seg_method,         # Vessel segmentation method
                                                          sigma1=sigma1,                # Sigma for enhacement filtering
                                                          hole_size=hole_size,          # Minimum hole size to fill
                                                          ditzle_size=ditzle_size,      # Small object removal threshold
                                                          thresh=thresh)                # Thresholding value
    seg_mask.append(vessel_seg) # Collect all vessel segmentations

#### <a id='toc3_3_1_3_'></a>[Vessel metric analysis and save results in Excel](#toc0_)

In [ ]:
all_roi_results = [] # Empty list to store ROI summary results
roi_raw = [] # Empty list to store ROI raw results
for i in range(len(seg_mask)): # Loop through each segmented ROI to calculate vessel metrics
    sampleID = output_ROI_paths[i].split("/")[-1].split('.tiff')[0] # Extract sample identifier
    sample_name = output_ROI_paths[i].split("/")[-1] # Extract sample full name
    label_path = os.path.join(output_path, f"{sampleID}_mask.tiff") # Path to the labelled mask file
    (length,   # List of raw vessel segment lengths (px)
     area,     # Vessel area (px²)
     conn,     # Connectivity
     Q,        # Q parameter
     jaccard,  # Jaccard similarity
     label,    # Labelled mask
     label_im  # Image of the labelled mask
     ) = calculate_and_print_metrics(label_path, seg_mask) # Calculate ROI segment metrics
    (skel,          # Skeletonised image
     edges,         # List of segment edges
     edge_labels,   # Labelled edges
     branchpoints,  # List of branchpoints and coordinates
     coords,        # Segment coordinates
     endpoints      # Skeleton endpoints
     ) = analyze_skeleton(seg_mask, gray_image = all_grayROIs[i], file_name = sample_name) # Skeletonisation of vessel segments
    (diam_values,           # List of raw vessel diameters
     viz,                   # Visualisation of vessel diameters
     filtered_diam_values   # List of filtered diameters
     ) = analyze_vessel_diameters(mip_imageROI = all_mipROIs[i], seg_mask = seg_mask[i], edge_labels = edge_labels, 
                                                      gray_image = all_grayROIs[i], file_name = sample_name) # Calculate segment diameters
    (length,                        # List of raw vessel segment lengths (px)
     vessel_len,                    # Mean vessel length (px)
     tort,                          # List of vessel segment tortuosity
     vessel_tor,                    # Mean vessel tortuosity
     diam_values_um,                # List of raw segment diameters (microns)
     length_um,                     # List of raw vessel segment lengths (microns)
     filtered_length_df,            # List of filtered vessel segment lengths (px)
     filtered_length_um_df,         # List of filtered vessel segment lengths (microns) 
     filtered_diam_values_df,       # List of filtered segment diameters (px)
     filtered_diam_values_um_df     # List of filtered segment diameters (microns)
     ) = visualize_vessel_metrics(edge_labels, diam_values, scale = scale)
    (net_length,        # Overall network length (px)
     net_length_um,     # Overall network length (microns)
     density,           # Vessel density map
     density_array,     # Vessel density values
     overlay,           # Image of vessel density overlay
     vessel_density,    # Overall vessel density
     bp_density         # Optional: branchpoint density
     ) = analyze_vessel_network(mip_imageROI = all_mipROIs[i], seg_mask = seg_mask[i], edges = edges, label = label, 
                                          scale = scale, window = window_size)
    z_step = z # Z-step for Z-stack calculation (microns)
    # Attempt to extract Z-stack depth from sampleID
    try:
        z_str = [s for s in sampleID.split('_') if s.startswith('z')][0]  # z_str: string like 'z1-25'
        z_start, z_end = map(int, z_str[1:].split('-'))                   # Numbers for the start and end of the slices
        z_depth = (z_end - z_start) * z_step                              # Physical Z-stack depth in microns
        if z_depth == 0:
            z_depth = 1  # Avoid division by zero
    except Exception as e:
        print(f"Error extracting z-depth for {sampleID}: {e}") # Print error message if z-depth not defined in sample name
    # Dictionary of summary ROI measurements for export
    roi_dict = {
        "Sample ID": sampleID,
        "Area": area,
        "Connectivity": conn,
        "Q": Q,
        "Jaccard": jaccard,
        "Mean raw vessel length (px)": np.mean(length) if len(length)>0 else 0,
        "Mean raw vessel length (um)": np.mean(length_um) if len(length_um)>0 else 0,
        "Mean vessel length (px)": vessel_len,
        "Mean vessel length (um)": vessel_len/1.2044335377302655,  # example conversion
        "Mean raw vessel diameter (px)": np.mean(diam_values) if len(diam_values)>0 else 0,
        "Mean raw vessel diameter (um)": np.mean(diam_values_um) if len(diam_values_um)>0 else 0,
        "Mean vessel diameter (px)": np.mean(filtered_diam_values) if filtered_diam_values is not None else 0,
        "Mean vessel diameter (um)": np.mean(filtered_diam_values_um_df) if filtered_diam_values_um_df is not None else 0,
        "Vessel Tortuosity": vessel_tor,
        "Network Length (px)": net_length,
        "Network Length (um)": net_length_um,
        "Vessel density": vessel_density,
        "Branchpoints": len(branchpoints),
        "Z-stack depth (um)": z_depth,
        "Network Length (px) normalized": net_length / z_depth,
        "Network Length (um) normalized": net_length_um / z_depth,
        "Vessel density normalized": vessel_density / z_depth,
        "Branchpoints normalized": len(branchpoints) / z_depth
    }
    # Dictionary of raw ROI measurements for export
    roi_dict_raw = {
        "Sample ID": sampleID,
        "Raw vessel diameter (px)": diam_values,
        "Raw vessel diameter (um)": diam_values_um,
        "Raw vessel length (px)": length,
        "Raw vessel length (um)": length_um,
        "vessel diameter (px)": filtered_diam_values_df,
        "vessel diameter (um)": filtered_diam_values_um_df,
        "vessel length (px)": filtered_length_df,
        "vessel length (um)": filtered_length_um_df        
    }  
    all_roi_results.append(roi_dict) # list of summary metrics for all ROIs
    roi_raw.append(roi_dict_raw) # list of raw data for all ROIs
save_roi_results_to_excel(all_roi_results, roi_raw, output_path=output_path, output_name=output_name) # Save all ROI results to Excel

### <a id='toc3_3_2_'></a>[For double channel](#toc0_)

#### <a id='toc3_3_2_1_'></a>[Run samples in working directory](#toc0_)

In [ ]:
# Note: For comments, please see the section above

In [ ]:
file_names = [f for f in os.listdir(data_path) if f.startswith("kdrlmCh_flk1") and f.endswith(".czi")]

all_mipROIs = []
all_grayROIs = []
all_claheROIs= []
all_otsuROIs = []
output_ROI_paths = []
output_ROImask_paths = []
all_coords = []

for file_name in file_names:
    file_path = os.path.join(data_path, file_name)
    sampleID = file_name.split('_MaxInt')[0]
    print(f"\n--- Processing {sampleID} ---")
    (image,
     roi_masks,
     roi_coords,
     mip_image,
     gray_image,
     clahe_image,
     otsu_mask,
     output_mip_path,
     output_mask_path, 
     mip_imageROI, 
     roi_grayROI, 
     roi_claheROI, 
     roi_otsuROI, 
     output_mipROI_paths, 
     output_maskROI_paths
    ) = load_and_process_czi_doubleROI(file_path, file_name, output_path, roi = ROI_num, channel = channel)
    all_mipROIs.extend(mip_imageROI)
    all_grayROIs.extend(roi_grayROI)
    all_claheROIs.extend(roi_claheROI)
    all_otsuROIs.extend(roi_otsuROI)
    output_ROI_paths.extend(output_mipROI_paths)
    output_ROImask_paths.extend(output_maskROI_paths)
    all_coords.extend(roi_coords)
    if mip_imageROI is None:
            continue  # Skip to the next file if loading failed

#### <a id='toc3_3_2_2_'></a>[Generate vessel segmentation masks](#toc0_)

In [ ]:
for i in range(len(all_mipROIs)):
    sampleID = output_ROI_paths[i].split("/")[-1].split('.tiff')[0]
    sample_name = output_ROI_paths[i].split("/")[-1]
    vessel_seg, remove_list = segment_and_analyze_vessels(file_name=sample_name, 
                                                          image=all_mipROIs[i],
                                                          gray_image=all_grayROIs[i], 
                                                          clahe_image=all_claheROIs[i], 
                                                          otsu_mask=all_otsuROIs[i], 
                                                          output_path=output_path,
                                                          im_filter=seg_method, 
                                                          sigma1=sigma1, 
                                                          hole_size=hole_size, 
                                                          ditzle_size=ditzle_size, 
                                                          thresh=thresh)
    seg_mask.append(vessel_seg)

#### <a id='toc3_3_2_3_'></a>[Vessel metric analysis and save results in Excel](#toc0_)

In [ ]:
all_roi_results = [] # Empty list to store ROI summary results
roi_raw = [] # Empty list to store ROI raw results
for i in range(len(seg_mask)): # Loop through each segmented ROI to calculate vessel metrics
    sampleID = output_ROI_paths[i].split("/")[-1].split('.tiff')[0] # Extract sample identifier
    sample_name = output_ROI_paths[i].split("/")[-1] # Extract sample full name
    label_path = os.path.join(output_path, f"{sampleID}_mask.tiff") # Path to the labelled mask file
    (length,   # List of raw vessel segment lengths (px)
     area,     # Vessel area (px²)
     conn,     # Connectivity
     Q,        # Q parameter
     jaccard,  # Jaccard similarity
     label,    # Labelled mask
     label_im  # Image of the labelled mask
     ) = calculate_and_print_metrics(label_path, seg_mask) # Calculate ROI segment metrics
    (skel,          # Skeletonised image
     edges,         # List of segment edges
     edge_labels,   # Labelled edges
     branchpoints,  # List of branchpoints and coordinates
     coords,        # Segment coordinates
     endpoints      # Skeleton endpoints
     ) = analyze_skeleton(seg_mask, gray_image = all_grayROIs[i], file_name = sample_name) # Skeletonisation of vessel segments
    (diam_values,           # List of raw vessel diameters
     viz,                   # Visualisation of vessel diameters
     filtered_diam_values   # List of filtered diameters
     ) = analyze_vessel_diameters(mip_imageROI = all_mipROIs[i], seg_mask = seg_mask[i], edge_labels = edge_labels, 
                                                      gray_image = all_grayROIs[i], file_name = sample_name) # Calculate segment diameters
    (length,                        # List of raw vessel segment lengths (px)
     vessel_len,                    # Mean vessel length (px)
     tort,                          # List of vessel segment tortuosity
     vessel_tor,                    # Mean vessel tortuosity
     diam_values_um,                # List of raw segment diameters (microns)
     length_um,                     # List of raw vessel segment lengths (microns)
     filtered_length_df,            # List of filtered vessel segment lengths (px)
     filtered_length_um_df,         # List of filtered vessel segment lengths (microns) 
     filtered_diam_values_df,       # List of filtered segment diameters (px)
     filtered_diam_values_um_df     # List of filtered segment diameters (microns)
     ) = visualize_vessel_metrics(edge_labels, diam_values, scale = scale)
    (net_length,        # Overall network length (px)
     net_length_um,     # Overall network length (microns)
     density,           # Vessel density map
     density_array,     # Vessel density values
     overlay,           # Image of vessel density overlay
     vessel_density,    # Overall vessel density
     bp_density         # Optional: branchpoint density
     ) = analyze_vessel_network(mip_imageROI = all_mipROIs[i], seg_mask = seg_mask[i], edges = edges, label = label, 
                                          scale = scale, window = window_size)
    z_step = z # Z-step for Z-stack calculation (microns)
    # Attempt to extract Z-stack depth from sampleID
    try:
        z_str = [s for s in sampleID.split('_') if s.startswith('z')][0]  # z_str: string like 'z1-25'
        z_start, z_end = map(int, z_str[1:].split('-'))                   # Numbers for the start and end of the slices
        z_depth = (z_end - z_start) * z_step                              # Physical Z-stack depth in microns
        if z_depth == 0:
            z_depth = 1  # Avoid division by zero
    except Exception as e:
        print(f"Error extracting z-depth for {sampleID}: {e}") # Print error message if z-depth not defined in sample name
    # Dictionary of summary ROI measurements for export
    roi_dict = {
        "Sample ID": sampleID,
        "Area": area,
        "Connectivity": conn,
        "Q": Q,
        "Jaccard": jaccard,
        "Mean raw vessel length (px)": np.mean(length) if len(length)>0 else 0,
        "Mean raw vessel length (um)": np.mean(length_um) if len(length_um)>0 else 0,
        "Mean vessel length (px)": vessel_len,
        "Mean vessel length (um)": vessel_len/1.2044335377302655,  # example conversion
        "Mean raw vessel diameter (px)": np.mean(diam_values) if len(diam_values)>0 else 0,
        "Mean raw vessel diameter (um)": np.mean(diam_values_um) if len(diam_values_um)>0 else 0,
        "Mean vessel diameter (px)": np.mean(filtered_diam_values) if filtered_diam_values is not None else 0,
        "Mean vessel diameter (um)": np.mean(filtered_diam_values_um_df) if filtered_diam_values_um_df is not None else 0,
        "Vessel Tortuosity": vessel_tor,
        "Network Length (px)": net_length,
        "Network Length (um)": net_length_um,
        "Vessel density": vessel_density,
        "Branchpoints": len(branchpoints),
        "Z-stack depth (um)": z_depth,
        "Network Length (px) normalized": net_length / z_depth,
        "Network Length (um) normalized": net_length_um / z_depth,
        "Vessel density normalized": vessel_density / z_depth,
        "Branchpoints normalized": len(branchpoints) / z_depth
    }
    # Dictionary of raw ROI measurements for export
    roi_dict_raw = {
        "Sample ID": sampleID,
        "Raw vessel diameter (px)": diam_values,
        "Raw vessel diameter (um)": diam_values_um,
        "Raw vessel length (px)": length,
        "Raw vessel length (um)": length_um,
        "vessel diameter (px)": filtered_diam_values_df,
        "vessel diameter (um)": filtered_diam_values_um_df,
        "vessel length (px)": filtered_length_df,
        "vessel length (um)": filtered_length_um_df        
    }  
    all_roi_results.append(roi_dict) # list of summary metrics for all ROIs
    roi_raw.append(roi_dict_raw) # list of raw data for all ROIs
save_roi_results_to_excel(all_roi_results, roi_raw, output_path=output_path, output_name=output_name) # Save all ROI results to Excel